In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [14]:
#请求URL
def get_page_content(request_url):
    #得到页面的内容
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html=requests.get(request_url,headers=headers,timeout=10)
    content = html.text
    # 通过content创建BeautifulSoup对象
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

In [15]:
def analysis(soup):
    temp = soup.find("div",class_ = 'search-result-list')
    #提取信息框中需要的信息
    car_name = temp.find_all(class_ = 'cx-name text-hover')
    car_price = temp.find_all(class_ = 'cx-price')
    car_picture = temp.find_all('img' )
    #创建DataFrame,将内容添加至df
    df=pd.DataFrame(columns=['名称','最低价格（万)','最高价格（万)','产品图片链接'])
    for i in range(len(car_name)):
        item={}
        item['名称']=car_name[i].text
        price = car_price[i].text
        #价格暂无时，赋值NaN
        if price == "暂无":
           item['最低价格（万)']= np.NaN
           item['最高价格（万)']= np.NaN
        elif '-' in price:
           item['最低价格（万)']=float(price.split('-')[0])
           item['最高价格（万)']=float(price.split('-')[1][:-1])  
        else:
           item['最低价格（万)']=float(price[:-1])
           item['最高价格（万)']=float(price[:-1]) 
        item['产品图片链接']='http:'+ car_picture[i]['src']
        df=df.append(item, ignore_index=True)
    return df

In [23]:
page_num=3
base_url='http://car.bitauto.com/xuanchegongju/?mid=8' 
result = pd.DataFrame(columns=['名称','最低价格（万)','最高价格（万)','产品图片链接'])
for i in range(page_num):
    request_url=base_url+ '&page='+ str(i+1)
    soup = get_page_content(request_url)
    df=analysis(soup)
    print(df)
    result = result.append(df)

D:\Anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
D:\Anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


         名称  最低价格（万)  最高价格（万)  \
0        宝来     9.88    15.60   
1        朗逸     9.99    16.19   
2       桑塔纳     8.69    11.59   
3        速腾    12.89    19.19   
4        迈腾    18.61    30.99   
5        探岳    18.59    31.39   
6        凌渡    14.29    19.69   
7       高尔夫    14.13    23.42   
8        途岳    16.58    22.38   
9       帕萨特    18.49    28.29   
10      途观L    21.88    31.18   
11       途昂    29.99    50.59   
12       途铠    11.49    15.99   
13       探歌    13.58    20.38   
14  一汽-大众CC    24.68    30.18   
15       探影    11.49    15.99   
16       途观    19.48    23.18   
17     Polo     9.99    12.39   
18      探岳X    23.58    29.58   
19       辉昂    34.00    63.90   

                                               产品图片链接  
0   http://img2.bitautoimg.com/autoalbum/files/201...  
1   http://img4.bitautoimg.com/autoalbum/files/201...  
2   http://img4.bitautoimg.com/autoalbum/files/201...  
3   http://img2.bitautoimg.com/autoalbum/files/201...  
4   http://img1.bitautoimg

D:\Anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [24]:
result.to_csv('price_volkswagen.csv',index=False,encoding='gbk')